In [ ]:
# %load_ext autoreload
# %autoreload 1
# %aimport metaheuristics.algorithme_genetique

In [1]:
cd ..

d:\Git\dir\ICO\codes


In [2]:
from metaheuristics.algorithme_genetique import*
from metaheuristics.base_problem import *
from loading_models import *
from vrptw import VRPTW
from copy import deepcopy

In [11]:
context = load_solomon('simple.csv', nb_cust=10, vehicle_speed=100)
# context = create_vrptw(CUSTOMER_DIR, DEPOTS_DIR, VEHICLES_DIR, DEPOTS_DISTANCES_DIR, CUSTOMER_DISTANCES_DIR, 2958047, MODE_VEHICLE="mean", vehicle_code=None)
vrptw = VRPTW(context)

In [12]:
cri=True
neighborhood_params={   'verbose':0,
                        'init_sol':'random',
                        'choose_mode':'random',
                        'use_methods' : ['intra_route_swap','inter_route_swap',
                            'intra_route_shift','inter_route_shift',
                            'two_intra_route_swap', 'two_intra_route_shift',
                            'delete_smallest_route'],
                        'force_new_sol': True,         
                        'full_search':True
                    }

instance =GeneticAlgorithm(progress_bar=False,num_evolu_per_search=200,neighborhood_params=neighborhood_params,reproductive_isolation=True,best_seed=True,rate_mutation=0.9)
instance.fit_search(vrptw)

[[0, 5, 7, 8, 9, 0], [0, 3, 6, 4, 10, 1, 2, 0]]

In [13]:
for _ in tqdm(range(10)):
    instance.search()

  0%|          | 0/10 [00:04<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "D:\miniconda\envs\ico\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Trump\AppData\Local\Temp\ipykernel_84044\1580366305.py", line 2, in <cell line: 1>
    instance.search()
  File "d:\Git\dir\ICO\codes\metaheuristics\algorithme_genetique.py", line 76, in search
    self.__evolution()
  File "d:\Git\dir\ICO\codes\metaheuristics\algorithme_genetique.py", line 220, in __evolution
    # for chromosome in self.population:
  File "d:\Git\dir\ICO\codes\metaheuristics\algorithme_genetique.py", line 202, in __regeneration
    for chromosome in self.population:
  File "d:\Git\dir\ICO\codes\metaheuristics\algorithme_genetique.py", line 98, in __generate_chromosome
    chromosome = N.initial_solution()
  File "d:\Git\dir\ICO\codes\neighborhood.py", line 50, in initial_solution
    init_sol = self.random_solution(nb_cust=len(self.context.customers) - 1)
  F

In [ ]:
n=instance.NEIGHBORHOOD()
n.set_params({'choose_mode':'delete_smallest_route'})
for _ in tqdm(range(10000)):

    s1=n.initial_solution()
    s2=n(s1)

    if not (s1.checker() and s2.checker()):
        print(s1)
        print(s2)
        print('1\n',s1.checker(),'\n1\n',s2.checker())
    
        break

In [ ]:
new_routes=S.routes
new_routes

In [ ]:
available_positions = [(i, j) for i in range(len(new_routes)) for j in range(1, len(new_routes[i]))]
available_positions

In [ ]:
import random
from copy import deepcopy
from solution import VRPTWSolution


In [ ]:
for _ in tqdm(range(10000)):
    S=n.initial_solution()
    s3=delete_random_route(S)

In [ ]:
def delete_random_route(solution: VRPTWSolution) -> VRPTWSolution:
    """
    Deletes the random route and inserts its customers in the other routes
    :param solution
    :return: Solution (or nothing)
    """
    routes = deepcopy(solution.routes)
    new_solution = None
    S = VRPTWSolution()
    is_sol = False
    n_iter = 0
    available_routes = list(range(len(routes)))

    while not is_sol and n_iter < 10 and available_routes:
        n_iter += 1
        deleted_index = random.choice(available_routes)
        deleted_route = routes.pop(deleted_index)
        n_cycle = 0
        new_routes = deepcopy(routes)
        available_positions = [(i, j) for i in range(len(new_routes)) for j in range(1, len(new_routes[i]))]
        while n_cycle < 10 and available_positions:
            n_cycle += 1
            #将被删除列表元素随机选取位置插入到其余列表
            #记录仪：recorder
            recorder_del=[]
            recorder_new_routes=[]
            recorder_available_positions=[]
            for i in range(1, len(deleted_route)-1):## not len(deleted_route)-2?

                recorder_del.append(deleted_route[i])
                if not available_positions: break
                recorder_available_positions.append(deepcopy(available_positions))
                r = random.choice(available_positions)
                r_route = r[0]
                r_pos = r[1]
                new_routes[r_route].insert(r_pos, deleted_route[i])
                recorder_new_routes.append(deepcopy(new_routes))
                available_positions.remove(r)
                
            is_sol = all((S.route_checker(route) for route in new_routes)) #!!!!
            is_is_sol=VRPTWSolution(new_routes).all_customers_checker()

            if is_sol and(not is_is_sol):
                new_sol = VRPTWSolution(new_routes)
                print('found')
                print('recorder_del')
                print(recorder_del)
                print('recorder_new_routes')
                print(recorder_new_routes)
                print('recorder_available_positions')
                print(recorder_available_positions)
                return new_sol

        routes.insert(deleted_index, deleted_route)
        available_routes.remove(deleted_index)
    new_sol = solution
    return new_sol

In [ ]:
for _ in range(10):
    instance.search()
    cri=cri and instance.best_solution.checker()
print(cri)

In [ ]:
for _ in range(500):
    N = vrptw.neighborhood({'verbose': 0, 'choose_mode': 'crossover'})
    s1, s2 = N.initial_solution(), N.initial_solution()
    s1, s2 = reduce_solution(s1, N), reduce_solution(s2, N)
    s3, s4 = N.get_neighbor_from_two(s1, s2)
    if not s3.checker() and s4.checker():
        
        print('cost s1 =', s1.cost(), 'cost s2 =', s2.cost())
        print("cost s1' =", s3.cost(), "cost s2' =", s4.cost())
        print('s1 equal s3 =', s1==s3)
        print('s2 equal s4 =', s2==s4)
        print('is s3 valid ?', s3.checker(), '| is s4 valid ?', s4.checker())
        print()
        print_different_route(s1, s3)
        print()
        print_different_route(s2, s4)

Concept of Nation

In [ ]:
context = load_solomon('simple.csv', nb_cust=10, vehicle_speed=100)
# context = create_vrptw(CUSTOMER_DIR, DEPOTS_DIR, VEHICLES_DIR, DEPOTS_DISTANCES_DIR, CUSTOMER_DISTANCES_DIR, 2958047, MODE_VEHICLE="mean", vehicle_code=None)

vrptw = VRPTW(context)
neighborhood_params={   'verbose':1,
                        'init_sol':'random',
                        'choose_mode':'random',
                        'force_new_sol': False,         
                        'full_search':True
                    }
instance =GeneticAlgorithm(num_evolu_per_search=1,neighborhood_params=neighborhood_params,reproductive_isolation=True,best_seed=True)
instance.fit_search(vrptw)

for _ in range(10):
    instance.search()
    print(instance.best_solution.checker())
instance.plot_evolution_cost()

In [ ]:
instance.problem.context.distances.shape

In [ ]:
instance.best_solution.checker()

In [ ]:
instance.best_solution

reproductive_isolation=False,best_seed=False

In [ ]:
instance.evolution_best_solution

reproductive_isolation=False,best_seed=True

In [ ]:
instance.evolution_best_solution

reproductive_isolation=True,best_seed=False

In [ ]:
instance.evolution_best_solution 

reproductive_isolation=True,best_seed=True

In [ ]:
instance.evolution_best_solution 

Exemple with load_solomon

In [ ]:
for _ in range(1):
    context = load_solomon('simple.csv', nb_cust=10, vehicle_speed=100)

# for route_id in customers['ROUTE_ID'].unique():
#     context = create_vrptw(CUSTOMER_DIR, DEPOTS_DIR, VEHICLES_DIR, DEPOTS_DISTANCES_DIR, CUSTOMER_DISTANCES_DIR, 2958047, MODE_VEHICLE="mean", vehicle_code=None)
   
    vrptw = VRPTW(context)
    neighborhood_params={   'verbose':0,
                            'init_sol':'random',
                            'choose_mode':'random',
                            'force_new_sol': True         
                        }
    instance =GeneticAlgorithm(num_evolu_per_search=10,neighborhood_params=neighborhood_params)
    instance.fit_search(vrptw)

    for _ in range(10):
        instance.search()
    instance.plot_evolution_cost()

DATA GENERATOR

In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
def generator_demand(low=8,high=42):
    tmp=int(np.random.normal(20,10))
    while(tmp<low or tmp>high):
        tmp=int(np.random.normal(20,10))
    return tmp

def generator_time_window(right=1300,avr_during=400,sigma_start=150,sigma_during=13):
    variance_start=sigma_start*sigma_start
    variance_during=sigma_during*sigma_during
    start=int(np.random.normal(450,variance_start))
    while(start>1050 or start<0):
        start=int(np.random.normal(450,variance_start))  
    
    during=int(np.random.normal(avr_during,variance_during))
    while(during<120):
        during=int(np.random.normal(avr_during,variance_during))  
    end=min(start+during,right)
    return start,end

def generator_datum():
    time_window = generator_time_window(1440, 650)
    start_time, end_time = time_window
    return random.randint(0, 100),random.randint(0, 100),generator_demand(),start_time,end_time,int(np.random.normal(80,3))

def generator_data(id_start,id_end):
    for i in range(id_start,id_end+1):
        print(f'{i},'+(str(generator_datum())).replace(' ','').replace('(','').replace(')','')+',')

In [ ]:
def generate_dataframe(n_customers=10):
    columns = ['CUSTNO', 'XCOORD', 'YCOORD', 'DEMAND', 'READYTIME', 'DUETIME', 'SERVICETIME']
    df = pd.DataFrame(data=([i]+list(generator_datum()) for i in range(1, n_customers+1)), columns=columns)
    df['CAPACITY'] = 200
    return df

def generate_csv(filename, n_customers):
    df = generate_dataframe(n_customers)
    df.to_csv(filename, index=False)    

In [ ]:
cd data_solomon

In [ ]:
generate_csv('A50.csv', 50)